In [1]:
from google.colab import drive
drive.mount('/content/drive')

import cv2
import numpy as np
import os
from glob import glob

INPUT_DIR = "/content/drive/MyDrive/BurnDetection_ProcessedDataset/augmented"
OUTPUT_DIR = "/content/drive/MyDrive/BurnDetection_ProcessedDataset/segmented"

os.makedirs(OUTPUT_DIR, exist_ok=True)

image_paths = glob(INPUT_DIR + "/*.jpg") + glob(INPUT_DIR + "/*.png")
print("Total images:", len(image_paths))

def grayscale_otsu_segmentation(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (5,5), 0)
    _, binary = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    return gray, binary

def hsv_otsu_segmentation(image):
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    h, s, v = cv2.split(hsv)

    # Otsu di kanal Value
    blur_v = cv2.GaussianBlur(v, (5,5), 0)
    _, v_otsu = cv2.threshold(blur_v, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    # Adaptive HSV Mask (rentang luka bakar umum)
    lower = np.array([0, 30, 50])
    upper = np.array([25, 255, 255])
    hsv_mask = cv2.inRange(hsv, lower, upper)

    combined = cv2.bitwise_and(v_otsu, hsv_mask)
    return combined

def morphological_processing(binary):
    kernel = np.ones((5,5), np.uint8)

    opening = cv2.morphologyEx(binary, cv2.MORPH_OPEN, kernel)
    closing = cv2.morphologyEx(opening, cv2.MORPH_CLOSE, kernel)

    # Hole Filling
    floodfill = closing.copy()
    h, w = closing.shape
    mask = np.zeros((h+2, w+2), np.uint8)
    cv2.floodFill(floodfill, mask, (0,0), 255)
    hole_filled = closing | cv2.bitwise_not(floodfill)

    return opening, closing, hole_filled

def contour_extraction(image, binary):
    contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contour_img = image.copy()
    cv2.drawContours(contour_img, contours, -1, (0, 255, 0), 2)
    return contour_img, contours

for path in image_paths:
    filename = os.path.basename(path)
    image = cv2.imread(path)

    # STEP 2.1: Grayscale + Otsu
    gray, binary_gray = grayscale_otsu_segmentation(image)

    # STEP 2.2: HSV + Otsu Value
    binary_hsv = hsv_otsu_segmentation(image)

    # STEP 2.3: Morfologi
    opening, closing, hole_filled = morphological_processing(binary_hsv)

    # STEP 2.4: Contour
    contour_result, contours = contour_extraction(image, hole_filled)

    # Simpan hasil
    cv2.imwrite(os.path.join(OUTPUT_DIR, "gray_" + filename), gray)
    cv2.imwrite(os.path.join(OUTPUT_DIR, "binary_gray_" + filename), binary_gray)
    cv2.imwrite(os.path.join(OUTPUT_DIR, "binary_hsv_" + filename), binary_hsv)
    cv2.imwrite(os.path.join(OUTPUT_DIR, "opening_" + filename), opening)
    cv2.imwrite(os.path.join(OUTPUT_DIR, "closing_" + filename), closing)
    cv2.imwrite(os.path.join(OUTPUT_DIR, "hole_filled_" + filename), hole_filled)
    cv2.imwrite(os.path.join(OUTPUT_DIR, "contour_" + filename), contour_result)

print("✅ Semua proses Segmentasi & Morfologi selesai!")

Mounted at /content/drive
Total images: 925
✅ Semua proses Segmentasi & Morfologi selesai!
